In [15]:
import pandas as pd
import math
import numpy as np
from sklearn import preprocessing, svm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from datetime import datetime

# df=quandl.get('WIKI/GOOGL') # Creating dataframe 
df=pd.read_csv('Project.csv',skiprows=1)
df=df[:-6]
df.tail()

df_dict=pd.read_csv('property_project.csv')
df_dict=df_dict[['address','lat','long']]
df_dict=df_dict.set_index('address').to_dict()
df_dict['lat']['JI XIANG COURT']

dates_dict={
    "Jan": 1,
    "Feb": 2,
    "Mar": 3,
    "Apr": 4,
    "May": 5,
    "Jun":6,
    "Jul":7,
    "Aug":8,
    "Sep":9,
    "Oct":10,
    "Nov":11,
    "Dec":12    
}

# Preprocessing
# -Converting Floor Area(1400 to 1500 -> 1450)
# -Monthly rental/ sqft> Monthly rental Psf
# -Building/Project Name , District -> lat, long
# -Lease Commencement Date -> date

def f(x):
    try:
        return sum([int(item) for item in str(x).split(' to ')])/2
    except:
        pass
    try:
        return int(str(x).split('>')[1])
    except:
        return 0
df['Floor Area (sq ft)'] = df['Floor Area (sq ft)'].apply(f)
df['Monthly Rent/ psf']=df['Monthly Gross Rent($)']/df['Floor Area (sq ft)']
df['lat']=df['Building/Project Name'].apply(lambda x:df_dict['lat'][x])
df['long']=df['Building/Project Name'].apply(lambda x:df_dict['long'][x])
def g(x):
    m,y=x.split('-')
    return pd.Timestamp(int(y),dates_dict[m],1).toordinal()
df['Lease Commencement Date']=df['Lease Commencement Date'].apply(g).astype(float)
# # Remove missing bedroom data
df['No. of Bedroom(for Non-Landed Only)']=df['No. of Bedroom(for Non-Landed Only)'].replace('na*',np.NaN)
df=df.dropna()

# # consolidaiton of Data
df=df[['Monthly Rent/ psf','lat','long','No. of Bedroom(for Non-Landed Only)','Floor Area (sq ft)','Lease Commencement Date']]
df['No. of Bedroom(for Non-Landed Only)']=df['No. of Bedroom(for Non-Landed Only)'].astype(float)
df['Floor Area (sq ft)']=df['Floor Area (sq ft)'].astype(float)

df.head(40)


,Monthly Rent/ psf,lat,long,No. of Bedroom(for Non-Landed Only),Floor Area (sq ft),Lease Commencement Date
0,3.120000,1.32,103.820487,3.0,1250.0,736938.0
1,3.466667,1.32,103.820487,2.0,750.0,736938.0
2,3.533333,1.32,103.820487,1.0,750.0,736938.0
4,4.923077,1.32,103.820487,1.0,650.0,736938.0
5,2.720000,1.32,103.820487,3.0,1250.0,736938.0
6,4.736842,1.32,103.820487,2.0,950.0,736938.0
7,4.466667,1.32,103.820487,1.0,750.0,736938.0
8,3.294118,1.32,103.820487,1.0,850.0,736938.0
9,2.608696,1.32,103.820487,2.0,1150.0,736938.0
10,5.043478,1.32,103.820487,2.0,1150.0,736938.0


In [23]:

X=np.array(df.drop(['Monthly Rent/ psf'],1))
Y=np.array(df['Monthly Rent/ psf'])
X = preprocessing.scale(X)
X_train, X_test, y_train, y_test= train_test_split(X,Y,test_size=0.2)
clf= LinearRegression()
clf.fit(X_train, y_train)
# print(X_train)
accuracy=clf.score(X_test,y_test)
clf,accuracy

(LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
 0.29340821268103257)